In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
df_sp = pd.read_csv(r'F:\Abhilah_First\Results\Observed_T\noised_stake_data.csv')
df_sp.head()

,Unnamed: 0,Julian Day,obs_period(days),elevation(m),Derbis thickness (cm),melt (cm),pdd,Model-1 Melt Rate (cm/day),Model-2 Melt Rate (cm/day),Model-3 Melt Rate (cm/day)
0,0,232,17,3868.0,110.112269,9.226452,186.225022,0.975783,0.530753,0.657459
1,1,250,18,3867.0,113.032401,9.876500,155.421463,0.760865,0.408535,0.509296
2,2,263,13,3867.0,114.128684,1.758438,110.102815,0.743350,0.397225,0.496367
3,3,273,10,3868.0,116.278306,3.607321,79.189979,0.689702,0.365156,0.458390
4,4,298,25,3868.0,112.753976,0.653328,128.831619,0.454563,0.244369,0.304456


In [2]:
df_model1_param = pd.read_csv(r'F:\Abhilah_First\Results\Observed_T\Parameters_1000.csv')
df_model1_param.head()

,Unnamed: 0,TF1 Model 1,TF2 Model 1,MSE Model 1,DDF Model 2,d0 Model 2,MSE Model 2,DDF Model 3,d0 Model 3,a Model 3,MSE Model 3
0,0,1.377083,-0.193208,0.628261,4.882812,12.876337,0.427875,4.344631,6.342217,0.352784,0.427687
1,1,1.029540,-0.380523,0.534095,4.590246,15.585462,0.478000,3.862951,10.103826,0.408360,0.473450
2,2,0.974426,-0.384808,0.469312,4.490768,14.961118,0.422020,3.831872,8.800334,0.382067,0.417324
3,3,1.065527,-0.355470,0.513937,4.848621,13.643871,0.424900,4.073580,8.720120,0.401118,0.421400
4,4,1.029322,-0.359518,0.516735,4.687397,13.981979,0.442138,4.059110,8.195743,0.390163,0.436305


In [3]:
data = {'DDF':df_model1_param['DDF Model 3']} 
df_all_iterations = pd.DataFrame(data)
df_all_iterations['d0'] = df_model1_param['d0 Model 3']
df_all_iterations['a'] = df_model1_param['a Model 3']

MSEs = list(df_model1_param['MSE Model 3'])
df_all_iterations['MSE'] = df_model1_param['MSE Model 3']
weight = []
for i in MSEs:
    weight.append(math.exp(-i))
df_all_iterations['Weight'] = weight
df_all_iterations.head()

,DDF,d0,a,MSE,Weight
0,4.344631,6.342217,0.352784,0.427687,0.652015
1,3.862951,10.103826,0.408360,0.473450,0.622850
2,3.831872,8.800334,0.382067,0.417324,0.658808
3,4.073580,8.720120,0.401118,0.421400,0.656128
4,4.059110,8.195743,0.390163,0.436305,0.646420


In [4]:
print(min(weight), max(weight))
count = 0
for i in weight:
    if i <= 0.655:
        count +=1
print('count: ', count) 
# there is no zero weight, number of non zero weights is 1000

0.5950928137112362 0.7122433546638144
count:  587


In [5]:
df_all_iterations['DDF*Weight'] = df_all_iterations['DDF']*df_all_iterations['Weight']
df_all_iterations['d0*Weight'] = df_all_iterations['d0']*df_all_iterations['Weight']
df_all_iterations['a*Weight'] = df_all_iterations['a']*df_all_iterations['Weight']

df_all_iterations['Weight*(DDF - xBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['DDF'] - sum(df_all_iterations['DDF*Weight']/1000) )**2
df_all_iterations['Weight*(d0 - yBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['d0'] - sum(df_all_iterations['d0*Weight']/1000) )**2
df_all_iterations['Weight*(a - zBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['a'] - sum(df_all_iterations['a*Weight']/1000) )**2

df_all_iterations

,DDF,d0,a,MSE,Weight,DDF*Weight,d0*Weight,a*Weight,Weight*(DDF - xBARw)^2,Weight*(d0 - yBARw)^2,Weight*(a - zBARw)^2
0,4.344631,6.342217,0.352784,0.427687,0.652015,2.832767,4.135223,0.230021,1.765471,0.608562,0.006213
1,3.862951,10.103826,0.408360,0.473450,0.622850,2.406039,6.293166,0.254347,0.843654,13.921478,0.014617
2,3.831872,8.800334,0.382067,0.417324,0.658808,2.524467,5.797728,0.251709,0.845336,7.724707,0.010609
3,4.073580,8.720120,0.401118,0.421400,0.656128,2.672789,5.721513,0.263185,1.239520,7.337066,0.013977
4,4.059110,8.195743,0.390163,0.436305,0.646420,2.623892,5.297896,0.252209,1.195604,5.139242,0.011780
...,...,...,...,...,...,...,...,...,...,...,...
995,4.620144,4.989309,0.319122,0.411968,0.662346,3.060132,3.304646,0.211369,2.444281,0.099099,0.002709
996,4.086537,7.628840,0.377830,0.421837,0.655841,2.680118,5.003305,0.247796,1.262448,3.328244,0.009868
997,4.023204,8.480927,0.390819,0.431081,0.649806,2.614303,5.510959,0.253957,1.139242,6.264044,0.011958
998,4.432102,5.201243,0.316135,0.386900,0.679159,3.010100,3.532470,0.214706,2.039670,0.020768,0.002525


In [6]:
ddf_variability = (sum(df_all_iterations['Weight*(DDF - xBARw)^2'])*(1000)/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)


d0_variability = (sum(df_all_iterations['Weight*(d0 - yBARw)^2'])*(1000)/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

a_variability = (sum(df_all_iterations['Weight*(a - zBARw)^2'])*(1000)/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

print(df_all_iterations['DDF'][MSEs.index(min(MSEs))], df_all_iterations['d0'][MSEs.index(min(MSEs))], df_all_iterations['a'][MSEs.index(min(MSEs))])
print(round(ddf_variability,3), round(d0_variability,3), round(a_variability,3))
print('RMSE =',min(MSEs)**0.5)

5.272702996330807 4.903786825408823 0.3637172174748475
0.047 0.114 0.005
RMSE = 0.5825252232384271
